In [20]:
#%%
from deltalake import DeltaTable, write_deltalake
import duckdb
import polars as pl


In [ ]:

table_path = "s3://deltalake-test/data/delta/sample_dataset"
minio_storage_options_rs = {
    "aws_access_key_id": "xII4v45kgcOz0nocIHgd",
    "aws_secret_access_key": "1muFKevSu1lvBv74ZZvIo27qo3k2mZkyTVRfzJNs",
    "aws_endpoint": "http://192.168.1.130:9000",
    "aws_region": "us-east-1",
    "allow_http": "true",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true"
}


## Writing

### Polars (native)

In [ ]:
df = pl.DataFrame(
    {"A":[1,2,3]}
).with_columns(pl.col("A").cast(pl.Float32))


df.write_delta(table_path, mode="overwrite", storage_options=minio_storage_options_rs)

In [8]:
dt = DeltaTable(table_path, storage_options=minio_storage_options_rs)

In [10]:
dt.history()

[{'timestamp': 1705248040032,
  'operation': 'CREATE TABLE',
  'operationParameters': {'protocol': '{"minReaderVersion":1,"minWriterVersion":2}',
   'mode': 'ErrorIfExists',
   'location': 's3://deltalake-test/data/delta/sample_dataset',
   'metadata': '{"configuration":{},"created_time":1705248040022,"description":null,"format":{"options":{},"provider":"parquet"},"id":"c99f3484-2537-44f8-833f-f663ad9d7209","name":null,"partition_columns":[],"schema":{"fields":[{"metadata":{},"name":"A","nullable":true,"type":"float"}],"type":"struct"}}'},
  'clientVersion': 'delta-rs.0.17.0',
  'version': 0}]

### Pandas (through pyarrow)

In [26]:
import pandas as pd
df_pd = pd.DataFrame(
    {"A": [1.5, 6, 55]}
).astype({"A": "float32"}) # arrow doesn't support "double"

In [ ]:
write_deltalake(dt, df_pd, mode="append")

In [28]:
dt.history()

[{'timestamp': 1705249275452,
  'operation': 'WRITE',
  'operationParameters': {'partitionBy': '[]', 'mode': 'Append'},
  'clientVersion': 'delta-rs.0.17.0',
  'version': 1},
 {'timestamp': 1705248040032,
  'operation': 'CREATE TABLE',
  'operationParameters': {'metadata': '{"configuration":{},"created_time":1705248040022,"description":null,"format":{"options":{},"provider":"parquet"},"id":"c99f3484-2537-44f8-833f-f663ad9d7209","name":null,"partition_columns":[],"schema":{"fields":[{"metadata":{},"name":"A","nullable":true,"type":"float"}],"type":"struct"}}',
   'protocol': '{"minReaderVersion":1,"minWriterVersion":2}',
   'mode': 'ErrorIfExists',
   'location': 's3://deltalake-test/data/delta/sample_dataset'},
  'clientVersion': 'delta-rs.0.17.0',
  'version': 0}]

In [32]:
write_deltalake(dt, df_pd.assign(A=(df_pd["A"]-1).astype("float32")), mode="append")

## Versioning

In [58]:
dt.history(limit=10)

[{'timestamp': 1705249653159,
  'operation': 'WRITE',
  'operationParameters': {'partitionBy': '[]', 'mode': 'Append'},
  'clientVersion': 'delta-rs.0.17.0',
  'version': 2},
 {'timestamp': 1705249275452,
  'operation': 'WRITE',
  'operationParameters': {'mode': 'Append', 'partitionBy': '[]'},
  'clientVersion': 'delta-rs.0.17.0',
  'version': 1},
 {'timestamp': 1705248040032,
  'operation': 'CREATE TABLE',
  'operationParameters': {'location': 's3://deltalake-test/data/delta/sample_dataset',
   'mode': 'ErrorIfExists',
   'metadata': '{"configuration":{},"created_time":1705248040022,"description":null,"format":{"options":{},"provider":"parquet"},"id":"c99f3484-2537-44f8-833f-f663ad9d7209","name":null,"partition_columns":[],"schema":{"fields":[{"metadata":{},"name":"A","nullable":true,"type":"float"}],"type":"struct"}}',
   'protocol': '{"minReaderVersion":1,"minWriterVersion":2}'},
  'clientVersion': 'delta-rs.0.17.0',
  'version': 0}]

In [ ]:
dt.to_pandas()

,A
0,1.0
1,2.0
2,3.0
3,1.5
4,6.0
5,55.0
6,0.5
7,5.0
8,54.0


In [41]:
dt.load_as_version(1)

In [42]:
dt.to_pandas()

,A
0,1.0
1,2.0
2,3.0
3,1.5
4,6.0
5,55.0


In [43]:
dt.load_as_version(0)

In [45]:
dt.to_pandas()

,A
0,1.0
1,2.0
2,3.0


In [66]:
dt.load_as_version('2024-01-14T16:33:09Z')
# bugged
# dt.load_as_version(datetime.datetime(2023,1,1))


In [67]:
dt.to_pandas()

,A
0,1.0
1,2.0
2,3.0
3,1.5
4,6.0
5,55.0
6,0.5
7,5.0
8,54.0


In [68]:

dt.load_as_version('2024-01-14T16:06:09Z')

In [69]:
dt.to_pandas()

,A
0,1.0
1,2.0
2,3.0


In [72]:
datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")

'2024-01-14T18:14:09Z'

In [73]:

dt.load_as_version(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"))

In [74]:
dt.to_pandas()

,A
0,1.0
1,2.0
2,3.0
3,1.5
4,6.0
5,55.0
6,0.5
7,5.0
8,54.0
